In [9]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor

dfTrain = pd.read_csv('../../../DataSet/trip_train.csv', parse_dates=['start_date', 'end_date'], infer_datetime_format=True)
dfTest = pd.read_csv('../../../DataSet/trip_test.csv', parse_dates=['start_date', 'end_date'], infer_datetime_format=True)
dfWeather = pd.read_csv('../../../DataSet/weather.csv', parse_dates=['date'], infer_datetime_format=True)
dfStation = pd.read_csv('../../../DataSet/station.csv')

In [10]:
dfTrain = dfTrain[['id', 'duration', 'start_date', 'start_station_id', 'subscription_type']]
dfTrain.head()

,id,duration,start_date,start_station_id,subscription_type
0,907649,396,2015-08-27 08:36:00,50,Subscriber
1,384043,636,2014-07-28 22:06:00,67,Subscriber
2,316176,334,2014-06-09 08:42:00,77,Subscriber
3,618874,666,2015-01-26 16:55:00,69,Subscriber
4,910977,318,2015-08-29 15:09:00,67,Subscriber


In [11]:
dates = {'year':dfTrain.start_date.dt.year,
         'month':dfTrain.start_date.dt.month, 
         'day':dfTrain.start_date.dt.dayofyear, 
         'weekday':dfTrain.start_date.dt.dayofweek, 
         'hour':dfTrain.start_date.dt.hour}

wdates = {'year':dfWeather.date.dt.year, 
          'month':dfWeather.date.dt.month, 
          'day':dfWeather.date.dt.dayofyear}

dfTrain = pd.DataFrame(dates).join(dfTrain)
subscriptionTypes = dfTrain.subscription_type.unique()
dfTrain.subscription_type = dfTrain.subscription_type.astype('category', categories=subscriptionTypes).cat.codes

#dfWeather.join(pd.DataFrame())

In [12]:
dfTrain = dfTrain[['id', 'duration', 'month', 'weekday', 'day', 'hour', 'start_station_id', 'subscription_type']]

In [13]:
dfTrain.head()

,id,duration,month,weekday,day,hour,start_station_id,subscription_type
0,907649,396,8,3,239,8,50,0
1,384043,636,7,0,209,22,67,0
2,316176,334,6,0,160,8,77,0
3,618874,666,1,0,26,16,69,0
4,910977,318,8,5,241,15,67,0


In [14]:
def cityNameToZipCode(row) :

    if row.city == 'San Francisco' :
        return 94107

    if row.city == 'Redwood City' :
        return 94063

    if row.city == 'Palo Alto' :
        return 94301

    if row.city == 'Mountain View' :
        return 94041

    if row.city == 'San Jose' :
        return 95113


In [15]:
dfStation.loc[:,'city'] = dfStation.apply(cityNameToZipCode, axis=1)
dfStation.rename(columns={'id':'start_station_id', 'city':'city_zipcode'}, inplace=True)
dfStation = dfStation[['start_station_id', 'city_zipcode']]



In [16]:
dfTrain.merge(dfStation, on='start_station_id')

,id,duration,month,weekday,day,hour,start_station_id,subscription_type,city_zipcode
0,907649,396,8,3,239,8,50,0,94107
1,432022,4120,8,5,242,14,50,1,94107
2,224386,4675,3,6,82,16,50,1,94107
3,298292,1595,5,0,146,13,50,1,94107
4,133980,384,12,0,357,8,50,0,94107
5,568667,1336,12,6,341,12,50,0,94107
6,905612,1090,8,2,238,8,50,0,94107
7,548982,488,11,1,322,9,50,0,94107
8,577261,330,12,1,350,8,50,0,94107
9,759364,732,5,4,128,16,50,0,94107
